## Data Analytics Module 4 - Medallion Architecture with Delta Lake

### Loading CSV files into the Databricks File System (DBFS)

In [ ]:
%sh
rm -r /dbfs/spark_lab
mkdir /dbfs/spark_lab
wget -O /dbfs/spark_lab/2019.csv https://raw.githubusercontent.com/kuljotSB/DatabricksGenAIEngineer/refs/heads/main/Databricks_Fundamentals/2019_edited.csv
wget -O /dbfs/spark_lab/2020.csv https://raw.githubusercontent.com/kuljotSB/DatabricksGenAIEngineer/refs/heads/main/Databricks_Fundamentals/2020_edited.csv
wget -O /dbfs/spark_lab/2021.csv https://raw.githubusercontent.com/kuljotSB/DatabricksGenAIEngineer/refs/heads/main/Databricks_Fundamentals/2021_edited.csv
     

### Loading the CSV files into a Dataframe

In [ ]:
df = spark.read.load('spark_lab/*.csv', format='csv')
display(df.limit(100))

### Defining the schema for the Bronze Layer Dataframe

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

bronzeSchema = StructType([
    StructField("SalesOrderNumber", StringType()),
    StructField("SalesOrderLineNumber", IntegerType()),
    StructField("OrderDate", DateType()),
    StructField("CustomerName", StringType()),
    StructField("Email", StringType()),
    StructField("Item", StringType()),
    StructField("Quantity", IntegerType()),
    StructField("UnitPrice", FloatType()),
    StructField("Tax", FloatType())
])

### Creating the Bronze Layer Dataframe

In [ ]:
bronze_df = spark.read.load('/spark_lab/*.csv', format='csv', schema=bronzeSchema)
display(bronze_df.limit(100))

### Creating the Bronze Schema

In [ ]:
%sql
CREATE SCHEMA IF NOT EXISTS YOUR_UC_CATALOG_NAME.Bronze;

### Storing the Bronze Layer as a Delta Table

In [ ]:
from delta.tables import *
from pyspark.sql.functions import *

# Storing the Bronze Layer as a Delta Table in the Databricks File System (DBFS)
bronzeDeltaTablePath = '/delta/bronze_sales_orders'
bronze_df.write.format('delta').mode('overwrite').save(bronzeDeltaTablePath)

# Storing the Bronze Layer as a Delta Table in the Data Catalog
bronze_df.write.format('delta').saveAsTable('Bronze.bronze_sales_orders')

### Creating the Silver Layer by Cleaning and Transforming the Data

### Creating the Silver Layer Dataframe

In [ ]:
silver_df = spark.read.format("delta").load(bronzeDeltaTablePath)
silver_df.show(10)

### Cleaning the Silver Layer Dataframe

In [ ]:
from pyspark.sql.functions import col

silver_df = silver_df.dropDuplicates()
silver_df = silver_df.withColumn('Tax', col('UnitPrice') * 0.08)
silver_df = silver_df.withColumn('Tax', col('Tax').cast("float"))

### Creating the Silver Schema

In [ ]:
%sql
CREATE SCHEMA IF NOT EXISTS YOUR_UC_CATALOG_NAME.Silver;

### Storing the Silver Layer as a Delta Table

In [ ]:
from delta.tables import *
from pyspark.sql.functions import *

# Storing the Bronze Layer as a Delta Table in the Databricks File System (DBFS)
silverDeltaTablePath = '/delta/silver_sales_orders'
silver_df.write.format('delta').mode('overwrite').save(silverDeltaTablePath)

# Storing the Bronze Layer as a Delta Table in the Data Catalog
silver_df.write.format('delta').saveAsTable('Silver.silver_sales_orders')

### Creating the Gold Layer by Aggregating the Data

### Creating the Gold Layer Dataframe

In [ ]:
gold_df = spark.read.format("delta").load(silverDeltaTablePath)
gold_df.show(10)

### Aggregating Yearly Sales Data

In [ ]:
yearlySales = gold_df.select(year("OrderDate").alias("Year")).groupBy("Year").count().orderBy("Year")
display(yearlySales)

### Creating the Gold Schema in the UC

In [ ]:
%sql
CREATE SCHEMA IF NOT EXISTS YOUR_UC_CATALOG_NAME.Gold;

### Storing the Gold Layer as a Delta Table

In [ ]:
from delta.tables import *
from pyspark.sql.functions import *

# Storing the Bronze Layer as a Delta Table in the Databricks File System (DBFS)
goldDeltaTablePath = '/delta/gold_sales_orders'
yearlySales.write.format('delta').mode('overwrite').save(goldDeltaTablePath)

# Storing the Bronze Layer as a Delta Table in the Data Catalog
yearlySales.write.format('delta').saveAsTable('Gold.gold_sales_orders')